# Getting Started
Getting started with the dataset.

The objective of this notebook is to conduct an initial analysis of the dataset.<br>
Broken down, the objectives are:
- Parse data in the CSV
- Minimal EDA
- First model:
    - Evaluate binary classification vs multi-class classification

# Loading Data

In [1]:
import pandas as pd
import numpy as np

pd.set_option('mode.chained_assignment', None)

In [2]:
df = pd.read_csv("../data/raw/pl_23_24.csv")

Starting easy, with limited columns, for a preliminary analysis:<br>
Just keep the first seven columns. These columns contain basic match information<br>
- Date
- Time
- Home Team
- Away Team
- HG = Full Time Home Team Goals
- AG = Full Time Away Team Goals
- Res = Full Time Result (H=Home Win, D=Draw, A=Away Win)

In [3]:
def clean_df(input_df):
    """preliminary column selection"""
    df = input_df.copy()
    
    # Select columns
    cols_to_keep = df.columns[1:8]
    df = df[cols_to_keep]
        
    return df


In [4]:
df = clean_df(df)
df.columns

Index(['Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR'], dtype='object')

In [5]:
len(df)

203

In [6]:
df.head(3)

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,11/08/2023,20:00,Burnley,Man City,0,3,A
1,12/08/2023,12:30,Arsenal,Nott'm Forest,2,1,H
2,12/08/2023,15:00,Bournemouth,West Ham,1,1,D


In [7]:
df.tail(3)

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR
200,13/01/2024,17:30,Newcastle,Man City,2,3,A
201,14/01/2024,14:00,Everton,Aston Villa,0,0,D
202,14/01/2024,16:30,Man United,Tottenham,2,2,D


So, latest matches are at the bottom. Seems as results are appended to a list.

## Feature Eng
Creating time-related features. Based on the last 5 matches, calculate:
- Win rate
- Loss rate
- Goals scored
- Goals Conceded

This will probably require window functions.

Test. Get all games for an arbitrary team (in this case, Newcastle)

In [8]:
newcastle_df = df[(df.HomeTeam == "Newcastle") | (df.AwayTeam == "Newcastle")].iloc[-5:]

In [9]:
newcastle_df

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR
163,16/12/2023,15:00,Newcastle,Fulham,3,0,H
173,23/12/2023,15:00,Luton,Newcastle,1,0,H
178,26/12/2023,12:30,Newcastle,Nott'm Forest,1,3,A
196,01/01/2024,20:00,Liverpool,Newcastle,4,2,H
200,13/01/2024,17:30,Newcastle,Man City,2,3,A


Here I've got a a list of the last 5 games for Newcastle. I need to generalize this.

#### Getting a list of all competing teams 

In [10]:
df.HomeTeam.value_counts()

HomeTeam
Burnley             11
Everton             11
Newcastle           11
Chelsea             11
Man United          11
Liverpool           10
Crystal Palace      10
West Ham            10
Aston Villa         10
Tottenham           10
Wolves              10
Nott'm Forest       10
Fulham              10
Arsenal             10
Brentford           10
Sheffield United    10
Brighton            10
Luton               10
Man City             9
Bournemouth          9
Name: count, dtype: int64

In [11]:
teams = df.HomeTeam.value_counts().index.values.tolist()
teams

['Burnley',
 'Everton',
 'Newcastle',
 'Chelsea',
 'Man United',
 'Liverpool',
 'Crystal Palace',
 'West Ham',
 'Aston Villa',
 'Tottenham',
 'Wolves',
 "Nott'm Forest",
 'Fulham',
 'Arsenal',
 'Brentford',
 'Sheffield United',
 'Brighton',
 'Luton',
 'Man City',
 'Bournemouth']

In [12]:
len(teams)

20

### Generalizing
The idea is: For each match-day, generate a DF with all the data for the given time window. (in this case, for the last 5 matches). <br>

So, for each match-day, there will be a dataframe with a single record per team, with the columns being: win rate, loss rate, goals scored, goals conceded.<br>
predictions will be made based on this data.

In [13]:
def get_last_n_games_df(
    input_df: pd.DataFrame,
    team: str, 
    n_games: int = 5
) -> pd.DataFrame:
    """Based on an input dataframe, get info of the last n games for a given team"""
    
    df = input_df[(input_df.HomeTeam == team) | (input_df.AwayTeam == team)].iloc[-n_games:]

    return df

In [14]:
newcastle_df

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR
163,16/12/2023,15:00,Newcastle,Fulham,3,0,H
173,23/12/2023,15:00,Luton,Newcastle,1,0,H
178,26/12/2023,12:30,Newcastle,Nott'm Forest,1,3,A
196,01/01/2024,20:00,Liverpool,Newcastle,4,2,H
200,13/01/2024,17:30,Newcastle,Man City,2,3,A


In [15]:
get_last_n_games_df(input_df=df, team="Newcastle")

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR
163,16/12/2023,15:00,Newcastle,Fulham,3,0,H
173,23/12/2023,15:00,Luton,Newcastle,1,0,H
178,26/12/2023,12:30,Newcastle,Nott'm Forest,1,3,A
196,01/01/2024,20:00,Liverpool,Newcastle,4,2,H
200,13/01/2024,17:30,Newcastle,Man City,2,3,A


OK. DF generating function works OK

Now, calculate metrics based on this data.

In [16]:
def calculate_team_metrics(input_df: pd.DataFrame, team: str) -> dict:
    """Calculate metrics:
    win_rate, loss_rate, goals_scored, goals_conceded
    """
    n_games = len(input_df)
    
    # Get Home metrics
    h_df = input_df[input_df.HomeTeam == team]
    h_goals_scored = h_df.FTHG.sum()
    h_goals_conceded = h_df.FTAG.sum()
        # Wins = H, Losses = A
    h_df["win"] = h_df.FTR.apply(lambda X: 1 if X == "H" else 0)
    h_df["loss"] = h_df.FTR.apply(lambda X: 1 if X == "A" else 0)
    h_wins = h_df.win.sum()
    h_losses = h_df.loss.sum()
    
    # Get Away metrics
    a_df = input_df[input_df.AwayTeam == team]
    a_goals_scored = a_df.FTAG.sum()
    a_goals_conceded = a_df.FTHG.sum()
        # Wins = A, Losses = H
    a_df["win"] = a_df.FTR.apply(lambda X: 1 if X == "A" else 0)
    a_df["loss"] = a_df.FTR.apply(lambda X: 1 if X == "H" else 0)
    a_wins = a_df.win.sum()
    a_losses = a_df.loss.sum()

    # Get total metrics
    return_dict = {
        "win_rate": (h_wins + a_wins) / n_games,
        "loss_rate": (h_losses + a_losses) / n_games,
        "goals_scored": h_goals_scored + a_goals_scored,
        "goals_conceded": h_goals_conceded + a_goals_conceded
    }

    return return_dict

In [17]:
newcastle_df

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR
163,16/12/2023,15:00,Newcastle,Fulham,3,0,H
173,23/12/2023,15:00,Luton,Newcastle,1,0,H
178,26/12/2023,12:30,Newcastle,Nott'm Forest,1,3,A
196,01/01/2024,20:00,Liverpool,Newcastle,4,2,H
200,13/01/2024,17:30,Newcastle,Man City,2,3,A


In [18]:
newcastle_dict = calculate_team_metrics(input_df=newcastle_df, team="Newcastle")

In [19]:
newcastle_dict

{'win_rate': 0.2, 'loss_rate': 0.8, 'goals_scored': 8, 'goals_conceded': 11}

OK. calculate_team_metrics working OK. 

Keep thinking of features...<br>
Maybe... get last 7, 5, 3, 1 games. Then keep / discard based on EDA.

# Enrich dataset with these metrics
Create window function to sweep last n games

In [20]:
df.iloc[:0].isna().all().all()

True

In [21]:
df.columns

Index(['Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR'], dtype='object')

In [22]:
help(get_last_n_games_df)

Help on function get_last_n_games_df in module __main__:

get_last_n_games_df(input_df: pandas.core.frame.DataFrame, team: str, n_games: int = 5) -> pandas.core.frame.DataFrame
    Based on an input dataframe, get info of the last n games for a given team



In [23]:
N_GAMES = 5

for idx, row in df.iloc[100:102].iterrows():

    # Restrict df to past games relative to current game
    reduced_df = df.iloc[:idx]

    # get last n games for home and away teams
    home_analysis_df = get_last_n_games_df(
        input_df=reduced_df,
        team=row["HomeTeam"],
        n_games=N_GAMES
    )
    away_analysis_df = get_last_n_games_df(
        input_df=reduced_df,
        team=row["AwayTeam"],
        n_games=N_GAMES
    )

    # Calculate stats for home and away teams
    home_stats = calculate_team_metrics(
        input_df=home_analysis_df,
        team=row["HomeTeam"]
    )
    away_stats = calculate_team_metrics(
        input_df=away_analysis_df,
        team=row["AwayTeam"]
    )
    print(f"Stats:\n{home_stats}\n{away_stats}")

    # Create DF with calculated metrics
    home_stats_df = pd.DataFrame(data=home_stats, columns=home_stats.keys(), index=[idx])
    away_stats_df = pd.DataFrame(data=away_stats, columns=away_stats.keys(), index=[idx])

    team_stats_df = home_stats_df.join(away_stats_df, lsuffix='_home', rsuffix='_away')

    print(idx)
    print(row)
    print(team_stats_df)
    print()

Stats:
{'win_rate': 0.2, 'loss_rate': 0.4, 'goals_scored': 4, 'goals_conceded': 6}
{'win_rate': 0.6, 'loss_rate': 0.4, 'goals_scored': 5, 'goals_conceded': 6}
100
Date        04/11/2023
Time             12:30
HomeTeam        Fulham
AwayTeam    Man United
FTHG                 0
FTAG                 1
FTR                  A
Name: 100, dtype: object
     win_rate_home  loss_rate_home  goals_scored_home  goals_conceded_home  \
100            0.2             0.4                  4                    6   

     win_rate_away  loss_rate_away  goals_scored_away  goals_conceded_away  
100            0.6             0.4                  5                    6  

Stats:
{'win_rate': 0.4, 'loss_rate': 0.4, 'goals_scored': 8, 'goals_conceded': 6}
{'win_rate': 0.2, 'loss_rate': 0.6, 'goals_scored': 6, 'goals_conceded': 10}
101
Date        04/11/2023
Time             15:00
HomeTeam     Brentford
AwayTeam      West Ham
FTHG                 3
FTAG                 2
FTR                  H
Name: 101, dty

In [24]:
team_stats_df

,win_rate_home,loss_rate_home,goals_scored_home,goals_conceded_home,win_rate_away,loss_rate_away,goals_scored_away,goals_conceded_away
101,0.4,0.4,8,6,0.2,0.6,6,10


In [25]:
df.iloc[101]

Date        04/11/2023
Time             15:00
HomeTeam     Brentford
AwayTeam      West Ham
FTHG                 3
FTAG                 2
FTR                  H
Name: 101, dtype: object

In [26]:
b_test_df = get_last_n_games_df(
    input_df=df.iloc[:101],
    team="Brentford"
)

In [27]:
b_test_df

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR
52,23/09/2023,17:30,Brentford,Everton,1,3,A
67,01/10/2023,14:00,Nott'm Forest,Brentford,1,1,D
74,07/10/2023,15:00,Man United,Brentford,2,1,H
82,21/10/2023,15:00,Brentford,Burnley,3,0,H
91,28/10/2023,12:30,Chelsea,Brentford,0,2,A


In [28]:
calculate_team_metrics(
    input_df=b_test_df,
    team="Brentford"
)

{'win_rate': 0.4, 'loss_rate': 0.4, 'goals_scored': 8, 'goals_conceded': 6}

Window function OK!<br>
Now I have to generalize it. Apply to each row of the dataframe.<br>
Also, debug. There might be  afew issues with first rows returning empty DataFrames (cold start problem).

In [29]:
N_GAMES = 5

for idx, row in df.iloc[:50].iterrows():

    # Restrict df to past games relative to current game
    reduced_df = df.iloc[:idx]

    # get last n games for home and away teams
    home_analysis_df = get_last_n_games_df(
        input_df=reduced_df,
        team=row["HomeTeam"],
        n_games=N_GAMES
    )
    away_analysis_df = get_last_n_games_df(
        input_df=reduced_df,
        team=row["AwayTeam"],
        n_games=N_GAMES
    )

    # Calculate stats for home and away teams
    try:
        home_stats = calculate_team_metrics(
            input_df=home_analysis_df,
            team=row["HomeTeam"]
        )

        away_stats = calculate_team_metrics(
            input_df=away_analysis_df,
            team=row["AwayTeam"]
        )
    except ZeroDivisionError:
        # Handle exception and 
        stats = {'win_rate': np.nan, 'loss_rate': np.nan, 'goals_scored': np.nan, 'goals_conceded': np.nan}
        home_stats, away_stats = stats, stats
    
    # Create DF with calculated metrics
    home_stats_df = pd.DataFrame(data=home_stats, columns=home_stats.keys(), index=[idx])
    away_stats_df = pd.DataFrame(data=away_stats, columns=away_stats.keys(), index=[idx])

    team_stats_df = home_stats_df.join(away_stats_df, lsuffix='_home', rsuffix='_away')

    print(idx)
    print(row)
    print(team_stats_df)
    print()

0
Date        11/08/2023
Time             20:00
HomeTeam       Burnley
AwayTeam      Man City
FTHG                 0
FTAG                 3
FTR                  A
Name: 0, dtype: object
  win_rate_home loss_rate_home goals_scored_home goals_conceded_home  \
0           NaN            NaN               NaN                 NaN   

  win_rate_away loss_rate_away goals_scored_away goals_conceded_away  
0           NaN            NaN               NaN                 NaN  

1
Date           12/08/2023
Time                12:30
HomeTeam          Arsenal
AwayTeam    Nott'm Forest
FTHG                    2
FTAG                    1
FTR                     H
Name: 1, dtype: object
  win_rate_home loss_rate_home goals_scored_home goals_conceded_home  \
1           NaN            NaN               NaN                 NaN   

  win_rate_away loss_rate_away goals_scored_away goals_conceded_away  
1           NaN            NaN               NaN                 NaN  

2
Date         12/08/2023
Time 

In [30]:
df[:10]

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,11/08/2023,20:00,Burnley,Man City,0,3,A
1,12/08/2023,12:30,Arsenal,Nott'm Forest,2,1,H
2,12/08/2023,15:00,Bournemouth,West Ham,1,1,D
3,12/08/2023,15:00,Brighton,Luton,4,1,H
4,12/08/2023,15:00,Everton,Fulham,0,1,A
5,12/08/2023,15:00,Sheffield United,Crystal Palace,0,1,A
6,12/08/2023,17:30,Newcastle,Aston Villa,5,1,H
7,13/08/2023,14:00,Brentford,Tottenham,2,2,D
8,13/08/2023,16:30,Chelsea,Liverpool,1,1,D
9,14/08/2023,20:00,Man United,Wolves,1,0,H


## Apply to full DF

In [31]:
import os

In [32]:
files = [f for f in os.listdir("../data/raw/") if f.endswith("csv")] 
files.sort()

In [33]:
len(files)

5

#### Load all data into Dataset

In [34]:
dfs_accum = []
for f in files:
    df_to_load = pd.read_csv(f"../data/raw/{f}")
    df_to_load = clean_df(df_to_load)
    dfs_accum.append(df_to_load)

In [35]:
df = pd.concat(dfs_accum).reset_index(drop=True)

In [36]:
df

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,09/08/2019,20:00,Liverpool,Norwich,4,1,H
1,10/08/2019,12:30,West Ham,Man City,0,5,A
2,10/08/2019,15:00,Bournemouth,Sheffield United,1,1,D
3,10/08/2019,15:00,Burnley,Southampton,3,0,H
4,10/08/2019,15:00,Crystal Palace,Everton,0,0,D
...,...,...,...,...,...,...,...
1718,12/01/2024,19:45,Burnley,Luton,1,1,D
1719,13/01/2024,12:30,Chelsea,Fulham,1,0,H
1720,13/01/2024,17:30,Newcastle,Man City,2,3,A
1721,14/01/2024,14:00,Everton,Aston Villa,0,0,D


OK. So, I have about 1700 matches for the last 5 seasons.<br>
Other data sources: Matches for FA Cup

### Generate New Dataset

In [37]:
N_GAMES = 5
stats_df_accum = []

for idx, row in df.iterrows():

    # Restrict df to past games relative to current game
    reduced_df = df.iloc[:idx]

    # get last n games for home and away teams
    home_analysis_df = get_last_n_games_df(
        input_df=reduced_df,
        team=row["HomeTeam"],
        n_games=N_GAMES
    )
    away_analysis_df = get_last_n_games_df(
        input_df=reduced_df,
        team=row["AwayTeam"],
        n_games=N_GAMES
    )

    # Calculate stats for home and away teams
    try:
        home_stats = calculate_team_metrics(
            input_df=home_analysis_df,
            team=row["HomeTeam"]
        )

        away_stats = calculate_team_metrics(
            input_df=away_analysis_df,
            team=row["AwayTeam"]
        )
    except ZeroDivisionError:
        # Handle exception and 
        stats = {'win_rate': np.nan, 'loss_rate': np.nan, 'goals_scored': np.nan, 'goals_conceded': np.nan}
        home_stats, away_stats = stats, stats
    
    # Create DF with calculated metrics
    home_stats_df = pd.DataFrame(data=home_stats, columns=home_stats.keys(), index=[idx])
    away_stats_df = pd.DataFrame(data=away_stats, columns=away_stats.keys(), index=[idx])

    team_stats_df = home_stats_df.join(away_stats_df, lsuffix='_home', rsuffix='_away')

    stats_df_accum.append(team_stats_df)

In [38]:
stats_df = pd.concat(stats_df_accum)

/var/folders/d6/81q0g3zx24s11rmkx_sqrksw0000gn/T/ipykernel_30577/385248336.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  stats_df = pd.concat(stats_df_accum)


In [39]:
stats_df.head(30)

,win_rate_home,loss_rate_home,goals_scored_home,goals_conceded_home,win_rate_away,loss_rate_away,goals_scored_away,goals_conceded_away
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


OK. So I'll drop about 30 records... Not too bad

In [40]:
len(stats_df)

1723

In [41]:
len(df)

1723

In [42]:
enhanced_df = df.join(stats_df)

In [43]:
enhanced_df.head()

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,win_rate_home,loss_rate_home,goals_scored_home,goals_conceded_home,win_rate_away,loss_rate_away,goals_scored_away,goals_conceded_away
0,09/08/2019,20:00,Liverpool,Norwich,4,1,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10/08/2019,12:30,West Ham,Man City,0,5,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10/08/2019,15:00,Bournemouth,Sheffield United,1,1,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10/08/2019,15:00,Burnley,Southampton,3,0,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10/08/2019,15:00,Crystal Palace,Everton,0,0,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
enhanced_df.tail()

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,win_rate_home,loss_rate_home,goals_scored_home,goals_conceded_home,win_rate_away,loss_rate_away,goals_scored_away,goals_conceded_away
1718,12/01/2024,19:45,Burnley,Luton,1,1,D,0.2,0.6,5,8,0.4,0.6,10,11
1719,13/01/2024,12:30,Chelsea,Fulham,1,0,H,0.6,0.4,8,7,0.4,0.6,7,9
1720,13/01/2024,17:30,Newcastle,Man City,2,3,A,0.2,0.8,7,12,0.6,0.2,9,5
1721,14/01/2024,14:00,Everton,Aston Villa,0,0,D,0.4,0.6,6,8,0.6,0.2,9,7
1722,14/01/2024,16:30,Man United,Tottenham,2,2,D,0.2,0.6,4,9,0.8,0.2,13,7


In [45]:
enhanced_df.FTR.value_counts(normalize=True)

FTR
H    0.440511
A    0.333140
D    0.226349
Name: proportion, dtype: float64

Interesting... Home teams tend to win...

In [46]:
enhanced_df.to_csv("../data/version_01/pl_2019_2024_5gw.csv", index=True, header=True)

In [47]:
enhanced_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1723 entries, 0 to 1722
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Date                 1723 non-null   object 
 1   Time                 1723 non-null   object 
 2   HomeTeam             1723 non-null   object 
 3   AwayTeam             1723 non-null   object 
 4   FTHG                 1723 non-null   int64  
 5   FTAG                 1723 non-null   int64  
 6   FTR                  1723 non-null   object 
 7   win_rate_home        1707 non-null   float64
 8   loss_rate_home       1707 non-null   float64
 9   goals_scored_home    1707 non-null   object 
 10  goals_conceded_home  1707 non-null   object 
 11  win_rate_away        1707 non-null   float64
 12  loss_rate_away       1707 non-null   float64
 13  goals_scored_away    1707 non-null   object 
 14  goals_conceded_away  1707 non-null   object 
dtypes: float64(4), int64(2), object(9)
mem

In [48]:
test_read_df = pd.read_csv("../data/version_01/pl_2019_2024_5gw.csv", index_col=0)

In [49]:
test_read_df

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,win_rate_home,loss_rate_home,goals_scored_home,goals_conceded_home,win_rate_away,loss_rate_away,goals_scored_away,goals_conceded_away
0,09/08/2019,20:00,Liverpool,Norwich,4,1,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10/08/2019,12:30,West Ham,Man City,0,5,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10/08/2019,15:00,Bournemouth,Sheffield United,1,1,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10/08/2019,15:00,Burnley,Southampton,3,0,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10/08/2019,15:00,Crystal Palace,Everton,0,0,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1718,12/01/2024,19:45,Burnley,Luton,1,1,D,0.2,0.6,5.0,8.0,0.4,0.6,10.0,11.0
1719,13/01/2024,12:30,Chelsea,Fulham,1,0,H,0.6,0.4,8.0,7.0,0.4,0.6,7.0,9.0
1720,13/01/2024,17:30,Newcastle,Man City,2,3,A,0.2,0.8,7.0,12.0,0.6,0.2,9.0,5.0
1721,14/01/2024,14:00,Everton,Aston Villa,0,0,D,0.4,0.6,6.0,8.0,0.6,0.2,9.0,7.0


OK. DF Saved OK